In [11]:
import numpy
import xarray
import scipy
from matplotlib import pyplot
from matplotlib import cm
import datacube
from datacube.storage import masking
import gdal
import lccs_l3
import osr
import os
from shapely.geometry import box
import rasterio.merge
from rio_cogeo.profiles import cog_profiles
from rio_cogeo.cogeo import cog_translate

In [7]:
out_filename = "lccs/lccs_fiji_"
overview_resampling = 'nearest'


In [3]:
def combine_tiles(tiles_locations,filename):
    files = tiles_locations
    output_filename = filename+'_TEMP'+'.tif'
    sources = [rasterio.open(path) for path in files]
    dest, out_transform = rasterio.merge.merge(sources)

    with rasterio.open(files[0]) as src:
        profile = src.profile

    profile['transform'] = out_transform
    profile['width'] = len(dest[0][0])
    profile['height'] = len(dest[0])

    with rasterio.open(output_filename, 'w', **profile) as dst:
        dst.write(dest.astype(rasterio.int16))

    [os.remove(path) for path in files]
    return output_filename


In [4]:
def create_cog(input, output, overview_resampling, bidx):
    cogeo_profile = 'deflate'
    nodata = -1
    overview_level = 6
    overview_resampling = overview_resampling
    threads = 8
    output_profile = cog_profiles.get(cogeo_profile)
    output_profile.update(dict(BIGTIFF=os.environ.get("BIGTIFF", "IF_SAFER")))
    block_size = min(
        int(output_profile["blockxsize"]), int(output_profile["blockysize"])
    )

    config = dict(
        NUM_THREADS=threads,
        GDAL_TIFF_INTERNAL_MASK=os.environ.get("GDAL_TIFF_INTERNAL_MASK", True),
        GDAL_TIFF_OVR_BLOCKSIZE=os.environ.get("GDAL_TIFF_OVR_BLOCKSIZE", block_size),
    )

    cog_translate(
        src_path=input,
        dst_path=output,
        dst_kwargs=output_profile,
        indexes=bidx,
        nodata=nodata,
        web_optimized=False,
        add_mask=False,
        overview_level=overview_level,
        overview_resampling=overview_resampling,
        config=config,
        quiet=False
    )

In [5]:
tiles_locations = []

for i in range(1, 117):
    filename = out_filename+str(i)+'_TEMP.tif'
    if os.path.isfile(filename):
        tiles_locations.append(filename)

In [12]:


#uncogged_output_file = combine_tiles(tiles_locations,out_filename)
target_filename = out_filename+'.tif'
#as we have created this bands separately, band index (bidx) is always 0
bidx = 0
create_cog(uncogged_output_file, target_filename, overview_resampling, bidx)
os.remove(uncogged_output_file)

Reading input: lccs/lccs_fiji__TEMP.tif

Adding overviews...
Updating dataset tags...
Writing output to: lccs/lccs_fiji_.tif
